## Customer Generator
The purpose of this script is to mimic a strong customer base using fake names, emails, and so on. A store such as Clay & Craft have many customers in-store and online. Creating the insert statments for all customers in SQL would take a significant amount of time since it is not autogenerated like a real business. This tool will solve this problem.

In [ ]:
import random
from faker import Faker
from faker.exceptions import UniquenessException

fake = Faker()
Faker.seed(42)
random.seed(42)

def escape_sql(s):
    """ Helper function to replace single apostophes """

    return s.replace("'", "''")

def generate_customer():
    """ Main function to generate customer records """

    name = fake.name()
    first, last = name.split()[0], name.split()[-1] # Seperating first and last name for email
    email = f"{first.lower()}.{last.lower()}@example.com"
    address = fake.address().replace("\n", ", ") # Removing line break from address to keep on one line
    phone = fake.phone_number()

    # Returning all values back as a tuple & removing single quotes
    return (escape_sql(name), escape_sql(email), escape_sql(address), escape_sql(phone))

num_customers = 4233 # I'm wanting about 4,233 customers not including supply sales customers
output_path = "./insert_customers.sql"


with open(output_path, "w", encoding="utf-8") as f:
    f.write("INSERT INTO customers (name, email, address, phone) VALUES\n")
    for i in range(num_customers): # Call function for all customers
        name, email, address, phone = generate_customer()
        line = f"('{name}', '{email}', '{address}', '{phone}')"

        # Append with comma until end tuple then use semicolon
        if i < num_customers - 1:
            f.write(line + ",\n")
        else:
            f.write(line + ";\n")
            
print(f"Generated customer records in: {output_path}")


Generated 4233 customer records in: ./insert_customers.sql


## Product Instance Generator

In [ ]:
import random
from datetime import datetime, timedelta
import os

# Based on existing 'Products' table
products = {
    1: 'Ceramic Mug',
    2: 'Dinnerware Set (Plates & Bowls)',
    3: 'Ceramic Baking Dish',
    4: 'Travel Cup with Silicone Sleeve & Lid',
    5: 'Self-Watering Planter',
    6: 'Teatime Set'
}

# Ceramic artists and hire dates
ceramic_artists = {
    3: datetime(2019, 6, 10),  # Joan Silva
    4: datetime(2019, 9, 5),   # Milo Tran
    5: datetime(2021, 11, 22), # Leila Young
    6: datetime(2024, 1, 1)    # Ben Carr
}

# Number of product instances per year/ I want to mimic need for new hires and company growth
instance_counts_by_year = {
    2019: 98,
    2020: 235,
    2021: 572,
    2022: 987,
    2023: 2112,
    2024: 3500,
    2025: 4008
}

# Glaze and color types
glazes = ['Glossy', 'Matte', 'Satin', 'Crackle']
colors = ['White', 'Sand', 'Terracotta', 'Sage', 'Ochre', 'Moss', 'Charcoal', 'Rose', 'Blue', 'Copper', 'Grey']

# Helper function to make sure creation date for a product is not before the artist was hired
def random_date_in_year(year, artist_start):
    start_date = max(datetime(year, 1, 1), artist_start)
    end_date = datetime(year, 12, 31)
    return start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

output_path = "./insert_product_instances.sql"

# Main loop
with open(output_path, "w", encoding="utf-8") as f:
    f.write("INSERT INTO ProductInstances (product_id, created_by, date_created, glaze_type, color_variant) VALUES\n")
    
    first_entry = True  # to handle commas properly
    
    for year, count in instance_counts_by_year.items():
        for _ in range(count):
            # Choosing random product
            product_id = random.choice(list(products.keys()))
            
            # Artists for that year
            valid_artists = [aid for aid, start in ceramic_artists.items() if start <= datetime(year, 12, 31)]
            artist_id = random.choice(valid_artists)
            artist_start = ceramic_artists[artist_id]
            
            # Random date in year after artist start
            date_created = random_date_in_year(year, artist_start)
            
            # Random glaze and color
            glaze = random.choice(glazes)
            color = random.choice(colors)
            
            # Format SQL line
            line = f"({product_id}, {artist_id}, '{date_created.date()}', '{glaze}', '{color}')"
            
            # Add comma except for first entry or last line will get ';' at end
            if first_entry:
                f.write(line)
                first_entry = False
            else:
                f.write(",\n" + line)
    
    # End SQL statement
    f.write(";\n")

print(f"Generated product instances in: {output_path}")

Generated 4233 product instances in: ./insert_product_instances.sql


## Inventory Generator
I created this tool to generate realistic inventory records for all 11,512 product instances produced by the studio. Each instance is assigned a status (sold, returned, available, defective) and a location, with statuses weighted by year and artist to reflect realistic business patterns. Older items are mostly sold, while recent items show higher availability and occasional returns. Returned items from older years are more likely to be sold again. To simulate a defect issue, the tool accounts for Ben Carr, a new hire, whose products have a higher chance of being defective or returned. The tool uses employee hire dates to ensure each product instance aligns with its production year, producing a SQL INSERT file.

In [ ]:
import random

output_path = "./insert_inventory.sql"

total_instances = 11512  # Must match the total number of ProductInstances

# Helper for weighted random choice
def choose_weighted_status(weights):
    statuses = list(weights.keys())
    probs = list(weights.values())
    return random.choices(statuses, probs)[0]

# Need to load in which artist created which instance_id
# Reading in insert_product instance SQL file to identify the artist_id for each instance
instance_artists = []  # list to store the artist_id for each product instance

with open("./insert_product_instances.sql", "r", encoding="utf-8") as f:
    for line in f:
        if line.strip().startswith("("):
            # (product_id, artist_id, 'date', 'glaze', 'color'),
            parts = line.strip()[1:].split(",")
            artist_id = int(parts[1].strip())  # storing artist_id from parts
            instance_artists.append(artist_id)

# Prepending dummy instance so it's not off by 1, so instance_artists[1] = first instance's artist
instance_artists.insert(0, None)

with open(output_path, "w", encoding="utf-8") as f:
    f.write("INSERT INTO Inventory (instance_id, status, location) VALUES\n")

    # Looping through all instance_id to create 1 inventory row per instance_id in order
    for i in range(1, total_instances + 1):

        artist = instance_artists[i]   # looking up artist who created instance i

        # -----------------------------------------------------------------------------------------
        # GROUP LOGIC BEFORE instance_id 4005 (the total products created before Ben Carr is hired)
        # -----------------------------------------------------------------------------------------
        if i < 4004:

            if 1 <= i <= 1334:
                weights = {
                    'sold': 0.92,
                    'returned': 0.03,
                    'defective': 0.05,
                    'available': 0.00
                }
            elif 1335 <= i <= 2768:
                weights = {
                    'sold': 0.90,
                    'returned': 0.03,
                    'defective': 0.05,
                    'available': 0.02
                }
            elif 2769 <= i <= 4004:
                weights = {
                    'sold': 0.50,
                    'returned': 0.15,
                    'defective': 0.10,
                    'available': 0.25
                }

        # -----------------------------------------
        # GROUP LOGIC AT OR AFTER Ben Carr is hired
        # -----------------------------------------
        else:
            # -----------------------
            # 2024 RANGE: 4005-7759
            # -----------------------
            if 4005 <= i <= 7759:
                if artist == 6:  # Ben Carr in 2024
                    # Moderate defects, high returns, fewer sold, few available
                    weights = {
                        'defective': 0.30,
                        'returned': 0.40,
                        'sold': 0.20,
                        'available': 0.10
                    }
                else:  # All other artists in 2024
                    # More sold than available, very low returns, low defects
                    weights = {
                        'sold': 0.65,
                        'available': 0.30,
                        'returned': 0.03,
                        'defective': 0.02
                    }

            # -----------------------
            # 2025 RANGE: 7759+
            # -----------------------
            else:  # i >= 7759+
                if artist == 6:  # Ben Carr in 2025
                    # No returns, many defects, moderate sold, mostly available
                    weights = {
                        'available': 0.70,
                        'sold': 0.20,
                        'defective': 0.10,
                        'returned': 0.00
                    }
                else:  # All other artists in 2025
                    # Mostly available, moderate sold, no returns, almost no defects
                    weights = {
                        'available': 0.70,
                        'sold': 0.29,
                        'defective': 0.01,
                        'returned': 0.00
                    }
        # Choosing status based on the weighted table
        status = choose_weighted_status(weights)

        # Defective products are always stored separately
        if status == "defective":
            location = "Defective Bin"
        else:
            locations = ["Studio Shelf A", "Studio Shelf B", "Warehouse 1", "Display Case 1", "Display Case 2"]
            location = random.choice(locations)

        line = f"({i}, '{status}', '{location}')"

        if i < total_instances:
            f.write(line + ",\n")
        else:
            f.write(line + ";\n")

print(f"Generated inventory records in: {output_path}")

Generated 11512 inventory records in: ./insert_inventory.sql


## Supply Sales Generator
This script generates realistic supply sales SQL INSERT statements, simulating yearly growth from 2019–2025. It focuses on 12 repeat school and business customers, gradually increasing their activity, assigning random dates, supplies, quantities, and totals, and sorting sales by date.

In [ ]:
from datetime import datetime, timedelta
import random

# Dictionary to assign Supplier_id and price
supplies = {
    1: 18.50, 2: 24.00, 3: 28.00, 4: 32.00, 5: 34.00, 6: 30.00, 7: 31.00, 8: 32.00,
    9: 33.00, 10: 29.50, 11: 28.00, 12: 34.00, 13: 30.50, 14: 31.50, 15: 33.00,
    16: 35.00, 17: 30.00, 18: 12.99, 19: 6.50, 20: 45.00, 21: 3.50, 22: 4.25,
    23: 9.99, 24: 5.00, 25: 7.95, 26: 6.50, 27: 11.99, 28: 8.25, 29: 10.75,
    30: 6.25, 31: 12.00, 32: 38.00, 33: 49.00, 34: 2.50, 35: 1.20, 36: 4.75,
    37: 11.00, 38: 2.50, 39: 1.80, 40: 15.00, 41: 9.95, 42: 15.00, 43: 85.00,
    44: 24.50, 45: 12.00, 46: 65.00, 47: 18.00, 48: 32.00, 49: 22.00, 50: 110.00,
    51: 28.00, 52: 14.50, 53: 9.75
}

# Number of supply sales per year
base_count_by_year = {
    2019: 50, # Early years/first contracts
    2020: 120, # Gradual growth/ more school/business contracts
    2021: 250, # Moderate expansion
    2022: 400, # Steady increase
    2023: 650, # Steady increase and repeat contracts
    2024: 900, # High volume year with large contracts
    2025: 1100 # Slight increase/reaching a steady state
}

# I only want to capture businesses and schools that buy the supplies
repeat_customers = list(range(4235, 4246))  # IDs 4235-4246 correspond to the 12 fake schools/businesses that I manually added to the end of the customer insert file

output_file_path = "./insert_supply_sales.sql"


# --- Helpers ---
def random_date(start, end):
    """Generate random datetime between start and end."""
    return start + timedelta(days=random.randint(0, (end - start).days))


# Define gradual active customer growth per year
active_customers_by_year = {
    2019: repeat_customers[:5],   # first 5 customers
    2020: repeat_customers[:7],   # first 7 customers
    2021: repeat_customers[:9],   # first 9 customers
    2022: repeat_customers,       # all 12
    2023: repeat_customers,
    2024: repeat_customers,
    2025: repeat_customers
}

all_rows = []

for year, count in base_count_by_year.items():
    active_customers = active_customers_by_year[year]
    
    # Determine repeat ratio for returning customers (50–80%)
    repeat_ratio = min(0.5 + (year - 2019) * 0.1, 0.8)
    num_repeats = int(count * repeat_ratio)
    num_new = count - num_repeats

    # Pick repeat and new customers
    repeat_sales = random.choices(active_customers, k=num_repeats)
    new_sales = random.choices(active_customers, k=num_new)
    year_customers = repeat_sales + new_sales
    random.shuffle(year_customers)

    # Generate sale rows with random dates
    year_sales = []
    for i in range(count):
        sale_date = random_date(datetime(year, 1, 1), datetime(year, 12, 31))
        staff_id = 7 if (year < 2023 or sale_date < datetime(2023, 3, 20)) else 8
        supply_id = random.choice(list(supplies.keys()))
        quantity = random.randint(1, 10)
        total = round(supplies[supply_id] * quantity, 2)
        customer_id = year_customers[i]

        year_sales.append({
            "staff_id": staff_id,
            "supply_id": supply_id,
            "customer_id": customer_id,
            "sale_date": sale_date,
            "quantity": quantity,
            "total": total
        })

    # Sort sales by date
    year_sales.sort(key=lambda x: x["sale_date"])

    # Convert to SQL rows
    for sale in year_sales:
        all_rows.append(
            f"({sale['staff_id']}, {sale['supply_id']}, {sale['customer_id']}, '{sale['sale_date'].date()}', {sale['quantity']}, {sale['total']})"
        )

# --- Write SQL INSERT File ---
with open(output_file_path, "w", encoding="utf-8") as f:
    f.write("INSERT INTO SupplySales (staff_id, supply_id, customer_id, sale_date, quantity, total) VALUES\n")
    f.write(",\n".join(all_rows))
    f.write(";\n")

print(f"Generated supply sales records in: {output_file_path}")


## Sales Generator

In [ ]:
import random
from datetime import datetime, timedelta

# --- Configuration ---
inventory_sql_file = "./insert_inventory.sql"
instances_sql_file = "./insert_product_instances.sql"
output_file_path = "./insert_sales.sql"

customer_ids = list(range(1, 4235))
staff_ids = [7, 8]  # sales staff

status_weights = {
    'completed': 0.9,
    'returned': 0.1
}

# --- Helpers ---
def parse_inventory(file_path):
    inventory = {}
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line.startswith("("):
                parts = line[1:-2].split(",")
                instance_id = int(parts[0].strip())
                status = parts[1].strip().strip("'")
                inventory[instance_id] = status
    return inventory

def parse_instances(file_path):
    instances = {}
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line.startswith("("):
                parts = line[1:-2].split(",")
                instance_id = len(instances) + 1
                date_created = datetime.strptime(parts[2].strip().strip("'"), "%Y-%m-%d")
                instances[instance_id] = date_created
    return instances

def random_date_after(start_date, max_days=30):
    return start_date + timedelta(days=random.randint(0, max_days))

def random_created_before(sale_date, max_days_before=5):
    return sale_date - timedelta(days=random.randint(1, max_days_before))

# --- Main Generation ---
inventory = parse_inventory(inventory_sql_file)
instances = parse_instances(instances_sql_file)
eligible_instances = [iid for iid, status in inventory.items() if status in ('sold', 'returned')]

all_rows = []

for instance_id in eligible_instances:
    created_date = instances[instance_id]
    sale_date = random_date_after(created_date, max_days=365*3)
    customer_id = random.choice(customer_ids)
    staff_id = random.choice(staff_ids)
    unit_price = round(random.uniform(20.0, 150.0), 2)
    quantity = 1
    total = round(unit_price * quantity, 2)
    status = random.choices(list(status_weights.keys()), list(status_weights.values()))[0]

    return_date = None
    if status == 'returned':
        return_date = random_date_after(sale_date, max_days=30).date()

    created_at = random_created_before(sale_date)

    row = f"({customer_id}, {instance_id}, {staff_id}, '{sale_date.date()}', {unit_price}, {quantity}, {total}, '{status}'"
    row += f", '{return_date}'" if status == 'returned' else ", NULL"
    row += f", NULL, '{created_at.date()}')"  # fake created_at a few days before sale
    all_rows.append(row)

# --- Write SQL INSERT File ---
with open(output_file_path, "w", encoding="utf-8") as f:
    f.write("INSERT INTO Sales (customer_id, product_instance_id, staff_id, sale_date, unit_price, quantity, total, status, return_date, return_reason, created_at) VALUES\n")
    f.write(",\n".join(all_rows))
    f.write(";\n")

print(f"Generated sales records in: {output_file_path}")


Generated 6874 sales records in: ./insert_sales.sql


## Shipping Information Generator

In [ ]:
import random
from datetime import datetime, timedelta

# --- Configuration ---
sales_sql_file = "./insert_sales.sql"  # file containing generated Sales inserts
output_file_path = "./insert_shipping_info.sql"

fake_carriers = ["FastShip", "QuickDeliver", "ParcelPro", "ShipRight"]

# --- Helpers ---
def parse_sales(file_path):
    """Parse the sales file to get sale_id and sale_date."""
    sales = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line.startswith("("):
                parts = line[1:-2].split(",")
                sale_id = len(sales) + 1  # auto-increment
                sale_date = datetime.strptime(parts[3].strip().strip("'"), "%Y-%m-%d")
                sales.append({"sale_id": sale_id, "sale_date": sale_date})
    return sales

def random_tracking_number():
    return f"TRK{random.randint(10000000, 99999999)}"

# --- Main Generation ---
sales = parse_sales(sales_sql_file)

# Find the most recent sale date
latest_sale_date = max(s["sale_date"] for s in sales)

all_rows = []

for sale in sales:
    sale_id = sale["sale_id"]
    sale_date = sale["sale_date"]

    # Shipped date: either the same day or one day after sale
    shipped_date = sale_date + timedelta(days=random.choice([0,1]))

    # Shipping duration 2-7 days
    delivery_days = random.randint(2, 7)
    delivered_date = shipped_date + timedelta(days=delivery_days)

    # Determine status
    if delivered_date > latest_sale_date:
        status = "In transit"
        delivered_date_sql = "NULL"
    else:
        status = "Delivered"
        delivered_date_sql = f"'{delivered_date.date()}'"

    carrier = random.choice(fake_carriers)
    tracking_number = random_tracking_number()

    row = f"({sale_id}, '{carrier}', '{tracking_number}', '{shipped_date.date()}', {delivered_date_sql}, '{status}')"
    all_rows.append(row)

# --- Write SQL INSERT File ---
with open(output_file_path, "w", encoding="utf-8") as f:
    f.write("INSERT INTO ShippingInfo (sale_id, carrier, tracking_number, shipped_date, delivered_date, status) VALUES\n")
    f.write(",\n".join(all_rows))
    f.write(";\n")

print(f"Generated shipping info records in: {output_file_path}")


Generated 6874 shipping info records in: ./insert_shipping_info.sql


## Product Review Generator

In [ ]:
import random
from datetime import datetime, timedelta

# --- Configuration ---
sales_sql_file = "./insert_sales.sql"      # Sales table inserts
shipping_sql_file = "./insert_shipping_info.sql"  # Shipping info inserts
output_file_path = "./insert_product_reviews.sql"

# Example review comments
good_comments = [
    "Beautiful craftsmanship",
    "Exactly as pictured",
    "High quality and durable",
    "Loved the finish and color",
    "Perfect gift",
    "Well made",
    "Very satisfied",
    "Arrived quickly and in good condition",
    "Excellent texture and detail",
    "Colors are vibrant and true to photo",
    "Sturdy and feels premium",
    "Exceeded my expectations",
    "Lovely design, great quality",
    "Arrived safely, no damage",
    "Easy to use and display",
    "Looks perfect in my collection",
    "Great packaging, product intact",
    "Fantastic quality for the price",
    "Smooth finish and consistent glaze",
    "Beautifully handmade, very happy",
    "Impressive attention to detail",
    "Exactly what I wanted",
    "Superb finish and color",
    "Feels solid and well-crafted",
    "Perfect size and shape",
    "Very professional craftsmanship",
    "Colors match description perfectly",
    "Arrived faster than expected",
    "Exceeded my expectations in quality",
    "Love the handmade look",
    "Perfect addition to my home",
    "Well packaged and protected",
    "Looks even better in person",
    "Glaze is flawless and smooth",
    "Highly recommend this product",
    "The craftsmanship is outstanding",
    "Beautifully made and elegant",
    "Color and finish are perfect",
    "The product is very sturdy",
    "Exactly as described on the site",
    "Highly satisfied with this purchase",
    "I will definitely order again",
    "Very nice quality for the price",
    "Absolutely love this piece",
    "Perfectly balanced and finished",
    "Looks great in my living room",
    "Wonderful gift for a friend",
    "The finish is gorgeous",
    "Feels luxurious and solid",
    "Exceeded my expectations in detail",
    "Handmade quality is evident",
    "Love the smooth texture",
    "Arrived in perfect condition",
    "Crafted with care and precision",
    "Colors are vibrant and even",
    "Fits perfectly in my space",
    "Very happy with this purchase",
    "Would buy again without hesitation",
    "The details are exquisite",
    "Fantastic quality craftsmanship",
    "Looks amazing and professional",
    "The design is unique and beautiful",
    "Handmade and looks premium",
    "Glaze and texture are perfect",
    "Really well constructed",
    "This is a beautiful product",
    "Exceeded expectations in every way",
    "Absolutely love the color",
    "Perfect for my collection",
    "Well finished and smooth",
    "Looks exactly like the photo",
    "Arrived quickly and safely",
    "Wonderful quality for the price",
    "Craftsmanship is impressive",
    "A very elegant piece",
    "Strong, sturdy, and well-made",
    "Colors are true to description",
    "Very high-quality product",
    "So pleased with this item",
    "Beautifully designed and finished",
    "Exactly as expected",
    "Highly detailed and well-crafted",
    "Gorgeous handmade finish",
    "Would highly recommend to others",
    "Perfect for gifting",
    "The glaze is smooth and even",
    "Looks even better in real life",
    "Very happy with the color",
    "Top-notch quality",
    "Craftsmanship shows attention to detail",
    "Elegant and sophisticated",
    "Very pleased with the purchase",
    "Fits beautifully in my home",
    "Handmade quality is excellent",
    "Looks beautiful on display",
    "Glaze and color are flawless",
    "Professional quality for the price",
    "Well packaged and delivered safely",
    "Extremely happy with this purchase",
    "The product looks exactly like the picture",
    "Vibrant colors and well finished",
    "A wonderful handmade item",
    "Exceeded expectations in quality and design",
    "Very sturdy and durable",
    "Looks perfect on my shelf",
    "Absolutely satisfied with this product",
    "Beautiful piece, highly recommend",
    "The craftsmanship is top-notch",
    "Feels high-end and premium",
    "Perfect gift for someone special",
    "Gorgeous handmade item",
    "Very impressed with quality and finish",
    "Exactly what I was hoping for",
    "Handmade details are beautiful",
    "A joy to own and display",
    "The product is elegant and well-made"
]

bad_comments = [
    "Air pocket in clay",
    "Cracked during shipping",
    "Broke easily",
    "Glaze inconsistent",
    "Color not as pictured",
    "Surface scratched",
    "Defective finish",
    "Uneven texture",
    "Warped shape",
    "Chipped edges",
    "Not durable",
    "Fell apart after first use",
    "Poor quality craftsmanship",
    "Smaller than expected",
    "Flawed design"
]

# --- Helpers ---
def parse_sales(file_path):
    """Parse sales inserts to get sale_id, order_group_id, quantity, status"""
    sales = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line.startswith("("):
                parts = line[1:-2].split(",")
                sale_id = len(sales) + 1
                # order_group_id can be NULL
                og = parts[1].strip()
                order_group_id = int(og) if og != "NULL" else None
                status = parts[9].strip().strip("'")  # status column
                sales.append({"sale_id": sale_id, "order_group_id": order_group_id, "status": status})
    return sales

def parse_shipping(file_path):
    """Parse shipping inserts to get delivered dates for each sale_id"""
    shipping = {}
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line.startswith("("):
                parts = line[1:-2].split(",")
                sale_id = int(parts[0])
                delivered_date = parts[4].strip()
                if delivered_date == "NULL":
                    delivered_date = None
                else:
                    delivered_date = datetime.strptime(delivered_date.strip("'"), "%Y-%m-%d")
                shipping[sale_id] = delivered_date
    return shipping

# --- Main Generation ---
sales = parse_sales(sales_sql_file)
shipping = parse_shipping(shipping_sql_file)

all_rows = []

for sale in sales:
    sale_id = sale["sale_id"]
    status = sale["status"]

    # Decide randomly if this sale gets a review (e.g., 70% chance)
    if random.random() > 0.7:
        continue

    # Determine rating
    if status == "returned":
        rating = random.randint(1, 2)  # low rating for returned products
        comment = random.choice(bad_comments)
    else:
        # Mostly positive reviews, small chance of low rating
        if random.random() < 0.05:
            rating = random.randint(1, 3)
            comment = random.choice(bad_comments)
        else:
            rating = random.randint(4, 5)
            comment = random.choice(good_comments)

    # Determine submission date (after delivery date if exists, else after sale date)
    delivered_date = shipping.get(sale_id)
    base_date = delivered_date if delivered_date else datetime.today()
    submitted_date = base_date + timedelta(days=random.randint(1, 30))

    row = f"({sale_id}, {rating}, '{comment}', '{submitted_date.date()}')"
    all_rows.append(row)

# --- Write SQL INSERT File ---
with open(output_file_path, "w", encoding="utf-8") as f:
    f.write("INSERT INTO ProductReviews (sale_id, rating, comment, submitted_date) VALUES\n")
    f.write(",\n".join(all_rows))
    f.write(";\n")

print(f"Generated product review records in: {output_file_path}")
